# Merging

In this lecture we're going to address how you can bring multiple dataframe objects together, either by merging them horizontally, or by concatenating them vertically. Before we jump into the code, we need to address a little relational theory and to get some language conventions down. I'm going to bring in an image to help explain some concepts.

![Venn Diagram](merging1.png)


Ok, this is a Venn Diagram. A Venn Diagram is traditionally used to show set membership. For example, the circle on the left is the population of students at a university. The circle on the right is the population of staff at a university. And the overlapping region in the middle are all of those students who are also staff.  Maybe these students run tutorials for a course, or grade assignments, or engage in running research experiments.

So, this diagram shows two populations whom we might have data about, but there is overlap between those populations.

When it comes to translating this to pandas, we can think of the case where we might have these two populations as indices in separate DataFrames, maybe with the label of Person Name. When we want to join the DataFrames together, we have some choices to make. First what if we want a list of all the people regardless of whether they're staff or student, and all of the information we can get on them? In database terminology, this is called a full outer join. And in set theory, it's called a union. In the Venn diagram, it represents everyone in any circle.

Here's an image of what that would look like in the Venn diagram.

![Union](merging2.png)

In [1]:
:set -XOverloadedStrings
import qualified DataFrame as D

import Data.Text

default (Text)

staffDf = D.fromRows
            ["Name", "Role"] [ [D.toAny "Kelly", D.toAny "Director of HR"]
                             , [D.toAny "Sally", D.toAny "Course liasion"]
                             , [D.toAny "James", D.toAny "Grader"]
                             ]

studentDf = D.fromRows
            ["Name", "School"] [ [D.toAny "James", D.toAny "Business"]
                               , [D.toAny "Mike", D.toAny "Law"]
                               , [D.toAny "Sally", D.toAny "Engineering"]
                               ]

D.take 10 staffDf
D.take 10 studentDf

  -----------------------------  
| Name<br>Text |  Role<br>Text  |
| -------------|--------------- |
| Kelly        | Director of HR |
| Sally        | Course liasion |
| James        | Grader         |


  -----------------------------  
| Name<br>Text | School<br>Text |
| -------------|--------------- |
| James        | Business       |
| Mike         | Law            |
| Sally        | Engineering    |


In [2]:
D.fullOuterJoin ["Name"] staffDf studentDf

  -----------------------------------------------------------  
| Name<br>Text | School<br>Maybe Text |  Role<br>Maybe Text   |
| -------------|----------------------|---------------------- |
| Sally        | Just "Engineering"   | Just "Course liasion" |
| James        | Just "Business"      | Just "Grader"         |
| Mike         | Just "Law"           | Nothing               |
| Kelly        | Nothing              | Just "Director of HR" |


In [3]:
D.innerJoin ["Name"] staffDf studentDf

  ----------------------------------------------  
| Name<br>Text | School<br>Text |  Role<br>Text  |
| -------------|----------------|--------------- |
| James        | Business       | Grader         |
| Sally        | Engineering    | Course liasion |


In [4]:
staffDf |> D.leftJoin ["Name"] studentDf

  ----------------------------------------------------  
| Name<br>Text |  Role<br>Text  | School<br>Maybe Text |
| -------------|----------------|--------------------- |
| Kelly        | Director of HR | Nothing              |
| Sally        | Course liasion | Just "Engineering"   |
| James        | Grader         | Just "Business"      |


In [5]:
staffDf |> D.rightJoin ["Name"] studentDf

  -----------------------------------------------------  
| Name<br>Text | School<br>Text |  Role<br>Maybe Text   |
| -------------|----------------|---------------------- |
| James        | Business       | Just "Grader"         |
| Mike         | Law            | Nothing               |
| Sally        | Engineering    | Just "Course liasion" |


So what happens when we have conflicts between the DataFrames? Let's take a look by creating new staff and student DataFrames that have a location information added to them.

In [ ]:
staffDf = D.fromRows ["Name", "Role", "Location"]
                     [ [D.toAny "Kelly", D.toAny "Director of HR", D.toAny "State Street"]
                     , [D.toAny "Sally", D.toAny "Course liasion", D.toAny "Washington Avenue"]
                     , [D.toAny "James", D.toAny "Grader", D.toAny "Washington Avenue"]
                     ]
studentDf = D.fromRows ["Name", "School", "Location"]
                       [ [D.toAny "James", D.toAny "Business", D.toAny "1024 Billiard Avenue"]
                       , [D.toAny "Mike", D.toAny "Law", D.toAny "Fraternity House #22"]
                       , [D.toAny "Sally", D.toAny "Engineering", D.toAny "512 Wilson Crescent"]
                       ]

staffDf
studentDf

  -------------------------------------------------  
| Name<br>Text |  Role<br>Text  | Location<br>Text  |
| -------------|----------------|------------------ |
| Kelly        | Director of HR | State Street      |
| Sally        | Course liasion | Washington Avenue |
| James        | Grader         | Washington Avenue |


  ----------------------------------------------------  
| Name<br>Text | School<br>Text |   Location<br>Text   |
| -------------|----------------|--------------------- |
| James        | Business       | 1024 Billiard Avenue |
| Mike         | Law            | Fraternity House #22 |
| Sally        | Engineering    | 512 Wilson Crescent  |


In [8]:
staffDf |> D.leftJoin ["Name"] studentDf

  -------------------------------------------------------------------------------------------------------  
| Name<br>Text |  Role<br>Text  | Location<br>Text  | School<br>Maybe Text | Right_Location<br>Maybe Text |
| -------------|----------------|-------------------|----------------------|----------------------------- |
| Kelly        | Director of HR | State Street      | Nothing              | Nothing                      |
| Sally        | Course liasion | Washington Avenue | Just "Engineering"   | Just "512 Wilson Crescent"   |
| James        | Grader         | Washington Avenue | Just "Business"      | Just "1024 Billiard Avenue"  |


From the output, we can see there is an extra column Right_Location. Right_Location refers to the Location column in the right dataframe, which is student dataframe.

Before we leave merging of DataFrames, let's talk about joining on multiple columns. It's quite possible that the first name for students and staff might overlap, but the last name might not. In this case, we use a list of the multiple columns that should be used to join keys from both dataframes on the on parameter.

In [ ]:
staffDf = D.fromRows ["First Name", "Last Name", "Role"]
                     [ [D.toAny "Kelly", D.toAny "Desjardins", D.toAny "Director of HR"]
                     , [D.toAny "Sally", D.toAny "Brooks", D.toAny "Course liasion"]
                     , [D.toAny "James", D.toAny "Wilde", D.toAny "Grader"]
                     ]
studentDf = D.fromRows ["First Name", "Last Name", "School"]
                       [ [D.toAny "James", D.toAny "Hammond", D.toAny "Business"]
                       , [D.toAny "Mike", D.toAny "Smith", D.toAny "", D.toAny "Law"]
                       , [D.toAny "Sally", D.toAny "Brooks", D.toAny "Engineering"]
                       ]

D.innerJoin ["First Name", "Last Name"] staffDf studentDf

  ------------------------------------------------------------------------  
| First Name<br>Text | Last Name<br>Text | School<br>Text |  Role<br>Text  |
| -------------------|-------------------|----------------|--------------- |
| Sally              | Brooks            | Engineering    | Course liasion |


Joining dataframes through merging is incredibly common, and you'll need to know how to pull data from different sources, clean it, and join it for analysis. This is a staple not only of dataframe, but of database technologies as well.

If we think of merging as joining "horizontally", meaning we join on similar values in a column found in two dataframes then concatenating is joining "vertically", meaning we put dataframes on top or at the bottom of each other

Let's understand this from an example. You have a dataset that tracks some information over the years. And each year's record is a separate CSV and every CSV ofr every year's record has the exactly same columns. What happens if you want to put all the data, from all years' record, together? You can concatenate them.

Let's take a look at the US Department of Education College Scorecard data It has each US university's data on student completion, student debt, after-graduation income, etc. The data is stored in separate CSV's with each CSV containing a year's record Let's say we want the records from 2011 to 2013 we first create three dataframe, each containing one year's record.

In [ ]:
-- Runs out of memory for other datasets.